In [1]:
import sys
import os
import time
sys.path.append('..')

In [3]:
from awsdet.utils.misc.config import Config
from awsdet.utils.logger import get_root_logger
from awsdet.utils.runner.dist_utils import init_dist
from awsdet.datasets.data_generator import DataGenerator
from awsdet.datasets import build_dataset
from awsdet.datasets.loader.build_loader import build_dataloader
import tensorflow as tf
import matplotlib.pyplot as plt

from awsdet.datasets.data_generator import DataGenerator

In [4]:
#config_path = '/workspace/shared_workspace/deep-learning-models/configs/rpn/rpn_r50_fpn_1x_coco.py'
config_path = '/workspace/shared_workspace/deep-learning-models/configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py'
cfg = Config.fromfile(config_path)
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
cfg.work_dir = os.path.join('./work_dirs',
                                os.path.splitext(os.path.basename(config_path))[0])
log_file = os.path.join(cfg.work_dir, f'{timestamp}.log')

In [5]:
init_dist()

In [6]:
coco = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=20.56s)
creating index...
index created!


In [7]:
#coco[0]['gt_masks'].to_tensor()

In [8]:
dg = DataGenerator(coco)()

In [9]:
img, meta, gt_boxes, gt_labels, gt_masks = next(dg)

Starting new loop for GPU: 0


In [10]:
train_tdf, train_size = build_dataloader(coco, 2)
train_tdf_iter = iter(train_tdf.repeat())

In [14]:
#%%timeit -r 10 -n 10
imgs, img_metas, gt_bboxes, gt_labels, gt_masks = next(train_tdf_iter)

In [15]:
imgs.shape

TensorShape([2, 1333, 1333, 3])

In [11]:
import matplotlib.patches as patches

def get_original_image(img):
    img = (img * \
       [58.395, 57.12, 57.375] + \
       [123.675, 116.28, 103.53])/255.
    #img = img[:img_instance['img_metas']['img_shape'][0], :img_instance['img_metas']['img_shape'][1], :]
    return img

def show_box(img, bbox):
    img = get_original_image(img)
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img)
    y1 = bbox[1]
    x1 = bbox[0]
    h = bbox[3] - bbox[1]
    w = bbox[2] - bbox[0]
    rect = patches.Rectangle((y1,x1),h,w,linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    plt.show()

In [12]:
#fig, ax = plt.subplots(figsize=(10, 10))
#ax.imshow(get_original_image(imgs[1]))

In [13]:
#fig, ax = plt.subplots(figsize=(10, 10))
#ax.imshow(gt_masks[1][0])

In [14]:
gt_labels

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[15, 15, 24, 28, 24, -1],
       [43, 45, 53, 42, 41, 60]], dtype=int32)>

In [15]:
#logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

In [16]:
coco.pipeline.transforms[-1].keys

['img', 'gt_bboxes', 'gt_labels', 'gt_masks']

In [17]:
#show_box(coco[0])

In [16]:
from awsdet.models.builder import build_backbone, build_neck

In [17]:
from awsdet.core.anchor.builder import build_anchor_generator
from awsdet.core.bbox.builder import build_assigner

In [18]:
#from awsdet.core.bbox.iou_calculators.iou2d_calculator import compute_overlaps
#from awsdet.utils.misc.meta import trim_zeros, calc_img_shapes
#from awsdet.core.bbox.assigners.assign_result

In [19]:
backbone = build_backbone(cfg.model.backbone)
neck = build_neck(cfg.model.neck)

In [22]:
%%timeit
imgs, img_metas, gt_bboxes, gt_labels, gt_masks = next(train_tdf_iter)
C2, C3, C4, C5 = backbone(imgs)
P2, P3, P4, P5, P6 = neck((C2, C3, C4, C5))

280 ms ± 51.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
anchor_generator = build_anchor_generator(cfg.model.rpn_head.anchor_generator)

In [24]:
rpn_feature_maps = [P2, P3, P4, P5, P6]

In [25]:
featmap_sizes = [tf.shape(i)[1:3] for i in rpn_feature_maps]
num_imgs = len(img_metas)
multi_level_anchors = anchor_generator.grid_anchors(featmap_sizes)
anchor_list = [multi_level_anchors for _ in range(num_imgs)]
valid_flags = [anchor_generator.valid_flags(featmap_sizes, i[3:5]) for i in img_metas]

In [26]:
assigner_dict = {'type': 'MaxIoUAssigner',
                 'pos_iou_thr': 0.7,
                 'neg_iou_thr': 0.3}
assigner = build_assigner(assigner_dict)

In [27]:
assign_result = assigner.assign(anchor_list[0], gt_bboxes[0], valid_flags=valid_flags[0], gt_labels=gt_labels[0])

In [28]:
gt_labels

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[ 2,  2,  7,  2,  7],
       [74, 74, -1, -1, -1]], dtype=int32)>

In [29]:
pos_anchors = tf.squeeze(tf.gather(tf.concat(anchor_list[0], axis=0), tf.where(assign_result.gt_inds>0)))
neg_anchors = tf.squeeze(tf.gather(tf.concat(anchor_list[0], axis=0), tf.where(assign_result.gt_inds==0)))

In [30]:
#tf.gather(result.labels, tf.where(result.gt_inds>0))

In [31]:
#show_box(imgs[0], pos_anchors[16])
import tensorflow as tf
from awsdet.utils.misc.meta import trim_zeros

In [32]:
anchors = tf.concat(anchor_list[0], axis=0)
gt_box, gt_bool = trim_zeros(gt_bboxes[0])
gt_flags = tf.zeros(tf.shape(anchors)[0], dtype=tf.int8)
bboxes = tf.concat([gt_box, anchors], axis=0)
gt_label = tf.boolean_mask(gt_labels[0], gt_bool)

In [33]:
self_inds = tf.range(
            1, len(gt_label) + 1, dtype=tf.int32)

In [34]:
gt_box

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 437.84964 ,    0.      ,  612.25134 ,  226.3053  ],
       [ 459.95743 , 1105.3916  ,  528.3126  , 1166.3893  ],
       [ 233.25014 ,   54.258667,  742.94073 , 1111.9185  ],
       [ 470.52667 , 1134.1655  ,  551.8184  , 1192.7389  ],
       [ 428.80893 ,    0.      ,  617.2843  ,  226.9953  ]],
      dtype=float32)>

In [35]:
anchors

<tf.Tensor: shape=(446118, 4), dtype=float32, numpy=
array([[ -22.627417,  -11.313708,   22.627417,   11.313708],
       [ -16.      ,  -16.      ,   16.      ,   16.      ],
       [ -11.313708,  -22.627417,   11.313708,   22.627417],
       ...,
       [ 917.9613  , 1098.9807  , 1642.0387  , 1461.0193  ],
       [1024.      , 1024.      , 1536.      , 1536.      ],
       [1098.9807  ,  917.9613  , 1461.0193  , 1642.0387  ]],
      dtype=float32)>

In [36]:
gt_label + 1

<tf.Tensor: shape=(13,), dtype=int32, numpy=array([26, 26, 10, 10,  1,  1, 29,  3,  3,  3, 10, 27, 10], dtype=int32)>

In [37]:
tf.unique_with_counts(assign_result.gt_inds)

UniqueWithCounts(y=<tf.Tensor: shape=(15,), dtype=int32, numpy=
array([ 0, -1, 11, 13,  9,  4,  3,  8,  2, 10,  7,  1, 12,  6,  5],
      dtype=int32)>, idx=<tf.Tensor: shape=(446118,), dtype=int32, numpy=array([0, 0, 0, ..., 1, 1, 1], dtype=int32)>, count=<tf.Tensor: shape=(15,), dtype=int32, numpy=
array([238029, 208065,      1,      1,      1,      4,      1,      2,
            1,      2,      4,      1,      1,      4,      1],
      dtype=int32)>)

In [39]:
assign_result.add_gt_(gt_label)

In [ ]:
assign_result.gt_inds

In [ ]:
assign_result.labels

In [35]:
# base dense head
from abc import ABCMeta, abstractmethod
import tensorflow as tf

class BaseDenseHead(tf.keras.Model, metaclass=ABCMeta):
    
    def __init__(self):
        super(BaseDenseHead, self).__init__()
    
    @abstractmethod
    def loss(self, **kwargs):
        """Coompute losses of the head"""
        pass
    
    @abstractmethod
    def get_bboxes(self, **kwargs):
        """Tranform network output for a batch into bbox predictions"""
        pass
    
    def forward_train(self, 
                         x, 
                         img_metas, 
                         gt_bboxes, 
                         gt_labels=None,
                         gt_bboxes_ignore=None,
                         proposal_cfg=None,
                         **kwargs):
        """
        Args:
            x (list[Tensor]): Features from FPN.
            img_metas (list[dict]): Meta information of each image, e.g,
                image size, scaling factor, etc.
            gt_bboxes (Tensor): Ground truth bboxes of teh image,
                shape (num_gts, 4).
            gt_labels (Tensor): Ground truth labels of each box,
                shape (num_gts,).
            gt_bboxes_ignore (Tensor): Ground truth bboxes to be
                ignored, shape (num_ignored_gts, 4).
            proposal_cfg (mmcv.Config): Test / postprocessing configuration,
                if None, test_cfg would be used
        Returns:
            tuple:
                losses: (dict[str, Tensor]): A dictionary of loss components.
                proposal_list (list[Tensor]): Proposals of each image.
        """
        outs = self(x)
        if gt_labels is None:
            loss_inputs = outs + (gt_bboxes, img_metas)
        else:
            loss_inputs = outs + (gt_bboxes, gt_labels, img_metas)
        losses = self.loss(*loss_inputs, gt_bboxes_ignore=gt_bboxes_ignore)
        if proposal_cfg is None:
            return losses
        else:
            proposal_list = self.get_bboxes(*outs, img_metas, cfg=proposal_cfg)
            return losses, proposal_list

In [37]:
tf.random.shuffle(tf.range(80))[:50]

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([62, 75, 11, 77,  0, 20,  1, 76, 27, 36, 55, 78, 59, 31, 53, 19, 63,
       52,  8, 50, 42, 10, 74, 39, 38, 17, 29, 16, 30, 60, 46, 34, 37, 21,
       28, 40, 23, 44, 25,  7, 73, 58, 48,  6,  5, 24, 66, 51, 64, 22],
      dtype=int32)>